# 🧠 RAG (Retrieval-Augmented Generation) Sistemi Adım Adım Rehber

Bu notebook, PDF dökümanlarınızı kullanarak özel bir yapay zeka asistanı eğitmek ve çalıştırmak için hazırlanmıştır. 

**Süreç:**
1.  Gerekli kütüphanelerin kurulumu
2.  Yapay zeka modellerinin hazırlanması
3.  Veri setinin (PDF'lerin) işlenmesi ve indekslenmesi
4.  Soru-cevap sisteminin testi

In [ ]:
# 1. GEREKSİNİMLERİ YÜKLEME
# Bu hücreyi çalıştırarak gerekli Python paketlerini kurun.
!pip install ollama chromadb pypdf tqdm fpdf

---

In [ ]:
# 2. OLLAMA MODELLERİNİ KONTROL ETME VE ÇEKME
# Sisteminizde Ollama kurulu olmalıdır. (https://ollama.com)
# Aşağıdaki komutlar modelleri indirir.

try:
    import ollama
    print("Mevcut Modeller:")
    models = ollama.list()
    for m in models['models']:
        print(f"- {m['name']}")
except Exception as e:
    print("Ollama bağlantı hatası. Lütfen Ollama uygulamasının çalıştığından emin olun.")
    print(f"Hata detayı: {e}")

# Eğer modeller yoksa terminalden şu komutları çalıştırın:
# !ollama pull nomic-embed-text
# !ollama pull RefinedNeuro/RN_TR_R2

---

### 3. İndeksleme (Eğitim) Aşaması
Bu adımda `docs/` klasöründeki PDF dosyaları okunacak, parçalanacak ve ChromaDB veritabanına vektör olarak kaydedilecektir.

In [ ]:
import os
import sys

# Proje ana dizinini yola ekle
current_dir = os.getcwd()
if current_dir not in sys.path:
    sys.path.append(current_dir)

from src import config
from src.ingest import run_indexing

print(f"Doküman Klasörü: {config.DOCS_DIR}")
print(f"Veritabanı Klasörü: {config.DB_DIR}")

# İndekslemeyi Başlat
if os.path.exists(config.DOCS_DIR) and os.listdir(config.DOCS_DIR):
    run_indexing()
else:
    print("UYARI: 'docs' klasörü boş veya mevcut değil. Lütfen içine PDF dosyaları ekleyin.")

---

### 4. Sohbet Aşaması
Artık indekslenen veriler üzerinde soru sorabilirsiniz.

In [ ]:
from src.chat import get_db, ask_question

# Veritabanı bağlantısı
collection = get_db()

if collection:
    soru = input("Sorunuzu yazın: ")
    if soru:
        print(f"\nAnaliz ediliyor: {soru}...")
        ask_question(collection, soru)
    else:
        print("Soru boş bırakıldı.")
else:
    print("Veritabanı yüklenemedi. Önce indeksleme yapın.")